In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/1/1.1_parse_sites_metadata/metadata.csv')

In [4]:
df

,pdb_code,site_name,site_idx,n_entities,n_atoms,n_bonds,metal,n_metals,n_waters,n_organic_ligands,n_metal_ligands,n_amino_acids,n_coordinating_amino_acids,n_nucleotides,non_residue_non_metal_names,n_non_residue_non_metal,coordination_distance
0,6fpw,6fpw_0,0,19,158,150,Fe,4,0,0,1,18,4,0,NaN,0,2.9
1,6fpw,6fpw_1,1,22,138,124,Fe,3,3,0,1,18,3,0,NaN,0,2.9
2,6fpw,6fpw_2,2,29,203,184,Fe,4,2,0,1,26,6,0,NaN,0,2.9
3,6fpw,6fpw_3,3,20,141,123,"Fe,Ni",2,0,0,1,19,4,0,NaN,0,2.9
4,6fpw,6fpw_4,4,19,101,83,Mg,1,6,0,1,12,3,0,NaN,0,2.9
5,6fpw,6fpw_5,5,19,158,150,Fe,4,0,0,1,18,4,0,NaN,0,2.9
6,6fpw,6fpw_6,6,23,146,131,Fe,3,3,0,1,19,3,0,NaN,0,2.9
7,6fpw,6fpw_7,7,29,203,184,Fe,4,2,0,1,26,6,0,NaN,0,2.9
8,6fpw,6fpw_8,8,20,141,123,"Fe,Ni",2,0,0,1,19,4,0,NaN,0,2.9
9,6fpw,6fpw_9,9,19,101,83,Mg,1,6,0,1,12,3,0,NaN,0,2.9


In [12]:
one_site = df.groupby('pdb_code').count()['site_name'] == 1

In [14]:
good_pdbs = list(one_site.index[one_site])

In [15]:
df = df[df['pdb_code'].isin(good_pdbs)]

In [18]:
df = df[df['n_metals'] == 1]

In [21]:
df[df['metal'].isin(['Co', 'Ni'])]

,pdb_code,site_name,site_idx,n_entities,n_atoms,n_bonds,metal,n_metals,n_waters,n_organic_ligands,n_metal_ligands,n_amino_acids,n_coordinating_amino_acids,n_nucleotides,non_residue_non_metal_names,n_non_residue_non_metal,coordination_distance
45,1rrk,1rrk_0,0,13,91,79,Co,1,0,1,0,12,3,0,CO,1,2.9
239,4fca,4fca_0,0,7,50,45,Ni,1,0,1,0,6,3,0,NI,1,2.9
1185,3sw8,3sw8_0,0,13,96,87,Ni,1,1,2,0,10,3,0,"5LI,NI",2,2.9
1553,7b9b,7b9b_0,0,8,65,62,Ni,1,0,1,0,7,4,0,NI,1,2.9
1744,2ga0,2ga0_0,0,6,44,39,Ni,1,0,1,0,5,3,0,NI,1,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122694,6h1x,6h1x_0,0,5,37,35,Co,1,0,1,0,4,3,0,CO,1,2.9
123368,2v24,2v24_0,0,7,60,58,Ni,1,0,1,0,6,3,0,NI,1,2.9
123405,5q60,5q60_0,0,10,73,68,Ni,1,1,2,0,7,4,0,"MLI,NI",2,2.9
123646,2ew5,2ew5_0,0,16,109,98,Co,1,4,2,0,10,3,0,"CO,Y12",2,2.9


In [25]:
df

,pdb_code,site_name,site_idx,n_entities,n_atoms,n_bonds,metal,n_metals,n_waters,n_organic_ligands,n_metal_ligands,n_amino_acids,n_coordinating_amino_acids,n_nucleotides,non_residue_non_metal_names,n_non_residue_non_metal,coordination_distance
15,1foi,1foi_0,0,13,85,72,Zn,1,0,1,0,12,4,0,ZN,1,2.9
23,2c2j,2c2j_0,0,11,71,60,Mg,1,0,1,0,10,3,0,MG,1,2.9
26,6bpv,6bpv_0,0,13,106,99,Fe,1,0,2,0,11,4,0,"F2Y,FE2",2,2.9
39,6tgt,6tgt_0,0,11,71,60,Ca,1,0,1,0,10,3,0,CA,1,2.9
45,1rrk,1rrk_0,0,13,91,79,Co,1,0,1,0,12,3,0,CO,1,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123930,2ghc,2ghc_0,0,15,99,87,Na,1,2,1,0,12,3,0,NaN,1,2.9
123949,4b0a,4b0a_0,0,9,69,62,Ca,1,0,1,0,8,3,0,CA,1,2.9
123955,4rvf,4rvf_0,0,13,92,81,Zn,1,0,1,0,12,4,0,ZN,1,2.9
123958,1tqs,1tqs_0,0,14,116,108,Zn,1,2,2,0,10,4,0,"SSO,ZN",2,2.9
